In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
def create_path(relative_path):
  return f'/content/drive/My Drive/MLofi/{relative_path}'

In [19]:
import os
from music21 import *
import numpy as np

In [20]:
import tensorflow as tf
from tensorflow import keras

In [21]:
pathToAllNotes = create_path('output/data/allNotes.txt')
with open(pathToAllNotes, "r") as f:
  allNotes = f.read().split(",")

In [22]:
uniqueNotes = set(allNotes)
mappingValuesToInt = dict()
mappingIntToValues = dict()
# LTSM works better with int values than with strings, so we need to map out notes (portrayed as strings) to ints
# (we also made a mapIntToVal, so once we need some values with our model, we can return them to their actual representation)
for i, val in enumerate(uniqueNotes):
    mappingValuesToInt[val] = i

    mappingIntToValues[i] = val

In [23]:
# For our model, we need to know the dimensions of our inputs and outputs, 
# that's why we are repeating this step
seqLen = 50

# inputs and outputs for our model
X_integer_encoded = []
Y_integer_encoded = []
    
for i in range(seqLen, len(allNotes)):
    
    inputValues = allNotes[i - seqLen : i]
    outputValues = allNotes[i]
    
    X_integer_encoded.append([mappingValuesToInt[note]  for note in inputValues]);
    Y_integer_encoded.append(mappingValuesToInt[outputValues]);

X_integer_encoded = np.reshape(X_integer_encoded, (len(X_integer_encoded), seqLen, 1))

In [24]:
X = keras.utils.to_categorical(X_integer_encoded)
Y = keras.utils.to_categorical(Y_integer_encoded)

In [25]:
noteCount = len(uniqueNotes)
notes = np.array(list(uniqueNotes))

r = np.random.randint(low=0, high=noteCount, size=seqLen)
rn = notes[r]
irn = [mappingValuesToInt[x] for x in rn]

In [28]:
# We are creating the same model we used as before, however, we aren't going to train it again,
# just insert the weights that are already saved!
model = keras.models.Sequential()
model.add(keras.layers.Input(shape=(X.shape[1], X.shape[2])))
model.add(keras.layers.LSTM(512, return_sequences=True))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.LSTM(512, return_sequences=True))
model.add(keras.layers.Dropout(0.3))
# ovaj lstm i dropout sam ja dodao!
model.add(keras.layers.LSTM(512, return_sequences=True))
model.add(keras.layers.Dropout(0.3))

model.add(keras.layers.LSTM(256))
# ovaj dropout sam ja dodao!
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(512))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(Y.shape[1]))
model.add(keras.layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [29]:
weightsPath = create_path("weights/weights.hdf5")
model.load_weights(weightsPath)

In [30]:
pattern = irn
prediction = []
for note_index in range(500):
    rp = np.reshape(np.array(pattern), (1, seqLen, 1))
    cp = keras.utils.to_categorical(rp, num_classes=noteCount)

    p = model.predict(cp, verbose=0)

    # print(p)
    i = np.argmax(p)
    result = mappingIntToValues[i]
    prediction.append(result)

    pattern.append(i)
    pattern = pattern[1:len(pattern)]
prediction

['E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',
 'E2.G2',


In [31]:
import re

In [32]:
ptn_1 = re.compile(r'[A-Z]')
ptn_2 = re.compile(r'Rest')

s = stream.Stream()
for element in prediction:
  if element == "padding":
    continue
  if ptn_2.search(element):
    s.append(note.Rest(element))
  elif ptn_1.search(element) is not None:
    s.append(note.Note(element))
  else:
    s.append(chord.Chord([int(x) for x in element.split('.')]))

AccidentalException: ignored

In [16]:
s.write('midi', fp=create_path('output/test/7.mid'))

'/content/drive/My Drive/MLofi/output/test/7.mid'